# Cython

Cython code must be compiled. This happens in two stages
1. A `.pyx` file is compiled by Cython to a `.c` file, containing code of a Python extension module.
2. The `.c` file is compiled by C compiler to a `.so` filewhich can be `imported` directly into a Python session

Cython is a Python compiler. It can compile the normal python code without changes (with a few exceptions of some as-yet unsupported features).

For performance it is helpful to **add static type declarations**, as they will tell Cython to step out of the dynamic nature of the Python code and generate simpler and faster C code.

## Example 1
Code in `hello_world.pyx`.

In [1]:
def say_hello(name):
    print(f'Hellow {name}!')

Now create `setup.py` and add this code.

In [ ]:
from setuptools import setup
from Cython.Build import cythonize

setup(ext_modules=cythonize('hello_world.pyx'))

Now run `python setup.py build_ext --inplace`. And you are done.

In [2]:
# Now you can import it
from hello_world import say_hello

In [3]:
say_hello('Kushaj')

Hellow Kushaj!


## Use Cython in Jupyter

In [4]:
%load_ext Cython

In [7]:
%%cython

cdef int a = 0
for i in range(10):
    a += i
print(a)

45


In [8]:
a

NameError: name 'a' is not defined

In [10]:
%%cython --annotate

# To show Cythons code analysis
cdef int a = 0
for i in range(10):
    a += i
print(a)

## Example 2
Code in `typing_variables.pyx`.

In [11]:
def f(x):
    return x**2 - x

def integrate_f(a,b,N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s*dx

Now we can make the above code better by adding type declarations.

In [ ]:
def f(double x):
    return x**2 - x

def integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s*dx

Declare the intermediate variables as they give the most speedup.

In [12]:
from typing_variables import *

In [14]:
f

<function typing_variables.f>

Now functions calls be expensive - in Cython doubly because one might need to convert to and from Python objects to do the call. In the above example, the argument of `f` is assumed to be double, yet a python float object must be constructed around the argument in order to pass it. **To avoid this use cdef for creating your functions that you will call**.

Also, some form of except-modifier should usually be added, otherwise Cython will not be able to propagate exceptions raised in the function (or a function that it calls). Also, you will not be able to call the cdef function as that is now not part of the python space.

In [ ]:
cdef double f(double x) except? -2:
    return x**2 - x

In [1]:
# Need to restart jupyter for this to work
from typing_variables import *

In [2]:
f

NameError: name 'f' is not defined

`cpdef` can be used if you also want a python wrapper. Syntax is same, it just adds a tiny overhead compared to `cdef`.

## Where to add types

Adding types to every variable cuts down on both readability and flexibility, and can even slow things down (e.g. by adding unnecessary type checks, conversions, or slow buffer unpacking).

Two tools you should use
1. Profiling
2. Annotation -> cython annotation can tell you why your code is taking time

Note that Cython deduces the types of local variables based on their assignments (including loop variables targets) which can also cut down on the need to explicitly specify types everywhere.

Use this command to use cython's annotation tool. `cython -a file.pyx`. This will generate a HTML report which you can view in chrome. 
* White lines -> translate to pure C code
* yellow lines -> require Python C-API.
* darker yellow -> more C-API interaction.

The white lines that don't interact with C will run as fast as C.

## Add C Code

To add C functions, we can import them from `from libc.stdlib cimport atoi`. The general syntax is `from libc.{C-library name} cimport {functions to import}`.

## Language Basics

`cdef` statement is used to declare C variables, either local or module-level.

In [1]:
%load_ext Cython

In [2]:
%%cython

cdef int i, j, k
cdef float f, g[42]

cdef struct Grail:
    int age
    float volume
    
cdef union Food:
    char *spam
    float *eggs
    
cdef enum CheeseState:
    hard=1
    soft=2
    runny=3
    
# To define a constant we can use `enum`
cdef enum:
    tons_of_spam = 3
    
print(tons_of_spam)

# Functions/Classes can also be defined with cdef
cdef class TempClass:
    cdef int width, height
    
    # These functions can also be initialized with cdef
    def __init__(self, w, h):
        self.width = w
        self.height = h
        
    def describe(self):
        pass

3


Cython uses the normal C syntax for C types, including pointers. It provides all the standard C types. `bint` is used for boolean integer types. Also, `cdef list`, `cdef dict`, `cdef tuple` can be used for static typing.

To define a tuple `cdef (double, int) bar`.

In [ ]:
%%cython

# Multiple things can be grouped in `cdef`
cdef:
    struct Spam:
        int tone
        
    int i
    float a
    

cdef (int, float) chips((long, log, double) t) except? -1: # It takes a tuple and returns a tuple
# except? -1 means that when this function return -1 it is possibly an exception
    
# If you want to return a python object, or use some argument that is a python object use `object`
cdef object chips(object int):

## Extension types

Cython lets you create new built-in Python types, known as **extension types**. Like when you used **cdef** to create a class. Also, you can use **cdef** to define the attributes. By default, extension type attributes are only accessible by direct access, not Python access, which means that they are not accessible from Python code. To make them accessible from Python code, you need to declare them as **public** or **readonly**. Like `cdef public int width, height`.

In your classes you can define **__cint__()** method where you should perform basic C-level initialization of the object, including allocation of any C data structures that your object will own.

## Sharing Declaration Between Cython Modules

A cython module can be split into two parts
1. `.pyd` -> definition file containing C declaration that are to be available to other Cython modules
2. `.pyx` -> implementation file

Use **cimport** to import.

In [ ]:
# dished.pxd
cdef enum otherstuff:
    sausage, eggs, lettuce

cdef struct spamdish:
    int oz_of_spam
    otherstuff filler

In [ ]:
# restaurent.pyx
cimport dishes
from dishes cimport spamdish

## Source files and compilations

Use `cython primes.pyx` to compile the file manually. It produces a `.c` file which then needs to be compiled with the C compiler using whatever options are appropriate on your platform for generating an extension module.

The other way is to use **setuptools** extensions provided with Cython. This will give the platform specific compilation options.

In [ ]:
from setuptools import setup
from Cython.Build import cythonize # Import it after setuptools

setup(
    name = "My hello app",
    ext_modules=cythonize("src/*.pyx", include_path=[numpy.get_include()]), # To get the necessary include files for numpy
)

# other way
# To compile with C++ pass language='c++' in cythonize comamnd
# To annotate pass `annotate=True`

## Early Binding for Speed

Consider this example.

In [5]:
%%cython

cdef class Rectangle:
    cdef int x0, y0
    cdef int x1, y1

    def __init__(self, int x0, int y0, int x1, int y1):
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1

    def area(self):
        area = (self.x1 - self.x0) * (self.y1 - self.y0)
        if area < 0:
            area = -area
        return area

def rectArea(x0, y0, x1, y1):
    rect = Rectangle(x0, y0, x1, y1)
    return rect.area()

There is a lot of overhead in the `rect.area()`. The code can be optimized as follows.

In [5]:
%%cython

cdef class Rectangle:
    cdef int x0, y0
    cdef int x1, y1
    
    def __init__(self, int x0, int y0, int x1, int y1):
        self.x0, self.y0, self.x1, self.y1 = x0, y0, x1, y1
        
    cdef int _area(self): # early bind
        area = (self.x1-self.x0)*(self.y1-self.y0)
        if area < 0: area = -area
        return area
    
    def area(self):
        return self._area()
    
def rectArea(x0, y0, x1, y1):
    cdef Rectangle rect = Rectangle(x0,y0,x1,y1) # early bind
    return rect._area()

## Fused Types

Multiple data types can be used to overloading of a variable.

In [9]:
%%cython

ctypedef fused char_or_float:
    char
    float
    
cdef char_or_float plus_one(char_or_float var):
    return var + 1

cdef:
    char a = 127
    float b = 127
print(f'{plus_one(a)}')
print(f'{plus_one(b)}')

-128
128.0


## Parallelism

First release the GIL.

In [1]:
%load_ext Cython

In [ ]:
%%cython

with nogil:
    <enter the code here>
    
# or use nogil=True

In [4]:
%%cython

from cython.parallel import prange

cdef:
    int i
    int n = 30
    int sum = 0
    
# for i in prange(n, nogil=True):
#     sum += i
    
for i in prange(0, n, 1, num_threads=8, nogil=True):
    # arguments same as range
    sum += i
    
print(sum)

435


In [ ]:
%%cython

from cython.parallel import parallel

with nogil, parallel(num_threads=8):
#     Run the below code on all 8 threads

To compile it, update the setup.py file as

In [ ]:
from setuptools import Extension, setup
from Cython.Build import cythonize

ext_modules = [
    Extension(
        "hellow",
        ["hello.pyx"],
        extra_compile_args=['-fopenmp'],
        extra_link_args=['fopenmp'],
    )
]

setup(
    name = 'hellow_parallel_world',
    ext_modules = cythonize(ext_modules),
)